# Plotting SEN2 data from a NetCDF

In [1]:
import rasterio
from rasterio.plot import show
from pathlib import Path

# Test should run from root
import os

os.chdir("..")

In [10]:
from omegaconf import OmegaConf
from metaearth.api import extract_assets
from metaearth.config import ConfigSchema

cfg = OmegaConf.load("config/sen2.yaml")
cfg_schema = OmegaConf.structured(ConfigSchema)
cfg = OmegaConf.merge(cfg_schema, cfg)


# Constrain the test to only pull one item and one band
for collection in cfg["collections"].keys():
    cfg["collections"][collection]["max_items"] = 1
    cfg["collections"][collection]["assets"] = ["B02"]     

successfully_extracted_assets, failed_assets = extract_assets(cfg)
print(
    f"Successfully extracted {len(successfully_extracted_assets)} assets. {len(failed_assets)} failed."
)

17:47:16 INFO - Extraction details for collection sentinel-2-l2a:
		provider=<Microsoft Planetary Computer (MPC)> 
		timerange=<2022-01-01/2022-01-02>,
		aoi_file=<config/aoi/demo.json>, 
		output_dir=<data>,
		assets=<['B02']>
17:47:16 INFO - 
Microsoft Planetary Computer (MPC) returned 1 items for sentinel-2-l2a for datetime 2022-01-01/2022-01-02

17:47:16 INFO - 1 assets did not specify file size, will query size directly with http get request (this may take a few moments)
system.query_asset_sizes=False can be used to disable this behavior
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.40it/s]
17:47:17 INFO - 0 assets have unknown file size
17:47:17 INFO - 1 assets to extract from 1 items
17:47:17 INFO - Asset types: 
key=B02; description="Band 2 - Blue - 10m"
17:47:17 INFO - Total asset size: 227 MB
17:47:17 INFO - Starting data extraction
17:47:17 INFO - Extracting 1 assets (227MB)...
MB:   0%|                                               | 0/227 [00:00<

Successfully extracted 1 assets. 0 failed.


In [8]:
# Find the first GeoTIFF file and load it
# We avoid hardcoding the path here in case the API changes down the road

fpath = Path(cfg["default_collection"]["outdir"])
collection_for_gt = "sentinel-2-l2a"
gt_files = list((fpath / collection_for_gt).glob("**/*.tif"))

assert (
    len(gt_files) <= 1
), "There were too many files pulled, `max_items` failed to constrain."



gt_file_fpath = gt_files[0]

data = rasterio.open(gt_file_fpath)

In [9]:
show(data)
print(data.crs)

ERROR 1: TIFFFillTile:Read error at row 4294967295, col 4294967295, tile 0; got 0 bytes, expected 344501
ERROR 1: TIFFReadEncodedTile() failed.
ERROR 1: data/sentinel-2-l2a/S2A_MSIL2A_20220101T184801_R070_T11SLC_20220102T161247/T11SLC_20220101T184801_B02_10m.tif, band 1: IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedTile() failed.


RasterioIOError: Read or write failed. data/sentinel-2-l2a/S2A_MSIL2A_20220101T184801_R070_T11SLC_20220102T161247/T11SLC_20220101T184801_B02_10m.tif, band 1: IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedTile() failed.